# Fine Tune Detection Model

In this model, we will show a very quick walkthrough on how to fine tune YOLOv8 using FiftyOne. We will go through how to load a dataset from HuggingFace, export the data to YOLO format, and then use Ultralytics to fine tune. At the end, we can easily apply our model to our entire dataset to view the results!

## Setup

If you haven't yet, install FiftyOne:

In [ ]:
!pip install fiftyone

We will also need some other libraries in order to train our YOLOv8 model:

In [ ]:
!pip install torch torchvision ultralytics

## Loading the Dataset

For our example workflow, I will be using the [VisDrone dataset](https://github.com/VisDrone/VisDrone-Dataset?tab=readme-ov-file), a state of the art drone imagery dataset from Lab of Machine Learning and Data Mining, Tianjin University, China. It features a wide range of locations, time of day, objects, and angles. We can download it directly from [HuggingFace](https://huggingface.co/datasets/Voxel51/VisDrone2019-DET) using the FiftyOne HuggingFace Hub integration!

In [ ]:
import fiftyone as fo
import fiftyone.utils.huggingface as fouh

# Load the dataset
dataset = fouh.load_from_hub("Voxel51/VisDrone2019-DET",overwrite=True)

# Launch the App
session = fo.launch_app(dataset)

![Visdrone](../assets/visdrone.png)

We need to export the dataset in YOLO format in order to train YOLOv8. We do so by randomly splitting and using the export method provided by the `dataset` class. This will allow ultralytics to automatically ingest our dataset without having to worry about converting any bounding boxes or labels manually!

In [ ]:
import fiftyone.utils.random as four

classes = dataset.distinct("ground_truth.detections.label")

four.random_split(dataset, {"val": 0.15, "train": 0.85})


for split in ["val","train","test"]:
    view =  dataset.match_tags(split)
    view.export(
        export_dir="VisDrone_YOLO/",
        split=split,
        dataset_type=fo.types.YOLOv5Dataset,
        classes=classes
    )

 100% |███████████████| 2804/2804 [21.3s elapsed, 0s remaining, 180.4 samples/s]      
Directory 'VisDrone_YOLO/' already exists; export will be merged with existing files
 100% |███████████████| 8578/8578 [1.1m elapsed, 0s remaining, 153.2 samples/s]      
Directory 'VisDrone_YOLO/' already exists; export will be merged with existing files
   6% |/--------------|  101/1610 [1.2s elapsed, 18.3s remaining, 82.6 samples/s]     

## Begin Training

We begin training using the Ultralytics library and passing in our exported `dataset.yaml`. I encourage you to try different hyperparameter setups and training configs based on your data. Check out all of your options [here](https://docs.ultralytics.com/modes/train/) at the Ultralytics Docs page! Once you have trained your model, you can pass it directly into `apply_model` to add the predicitions to your dataset!

In [ ]:
from ultralytics import YOLO


# Build a YOLOv9c model from pretrained weight
model = YOLO('yolov8n.pt')

# Display model information (optional)
model.info()

# Train the model on the  dataset for 1 epochs, set project name for experiment_name, name for the run name
model.train(
    data='VisDrone_YOLO/dataset.yaml',
    imgsz=640, 
    batch=1,
)

# Add predictions to our dataset
dataset.apply_model(model, label_field="YOLOv8")


![post-training](../assets/high_cf_fp.gif)

Great ML engineers don't stop here however! Check out other recipes on how to find annotation mistakes, evaluate your model, compare multiple models or more to make your model even better!